In [1]:
from pathlib import Path
import sys

# Assuming notebook is in project root
PROJECT_ROOT = Path("/scratch/lungerjo/DeepEmotion")
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

import os
import torch
from omegaconf import OmegaConf
from collections import Counter
from tqdm import tqdm
from srcutils.dataset import get_data_loaders
from models.CNN import CNN
from models.resnet import ResNet, BasicBlock

ModuleNotFoundError: No module named 'utils'

In [ ]:
!nvidia-smi

In [25]:
import os
import torch
import pandas as pd
from collections import Counter
from tqdm import tqdm
from omegaconf import OmegaConf

# Config variables
PROJECT_ROOT = os.path.abspath("../")

cfg = OmegaConf.create({
    "project_root": PROJECT_ROOT,
    "verbose": True,
    "wandb": True,
    "sys_log": True,
    "model": "CNN",
    "CNN": {
        "c1": 16, "c2": 32, "c3": 64, "k1": 3, "k2": 3, "k3": 3,
        "pk": 2, "ps": 2, "kernel_size": 3, "stride": 1, "padding": 1
    },
    "train": {
        "epochs": 50, "batch_size": 20, "shuffle": True, "train_ratio": 0.8,
        "print_label_frequencies": True
    },
    "data": {
        "data_path": f"{PROJECT_ROOT}/data/raw/derivatives/non-linear_anatomical_alignment",
        "zarr_dir_path": f"{PROJECT_ROOT}/zarr_datasets",
        "zarr_path": f"{PROJECT_ROOT}/zarr_datasets/pool_emotions",
        "label_path": f"{PROJECT_ROOT}/data/updated_annotations/pooled_annotations_structured.tsv",
        "sessions": ["01", "02", "03", "04", "05", "06", "07", "08"],
        "file_pattern_template": "*_ses-forrestgump_task-forrestgump_rec-dico7Tad2grpbold7TadNL_run-{}_bold.nii.gz",
        "subjects": ["sub-01"],
        "session_offsets": [0, 902, 1784, 2660, 3636, 4560, 5438, 6522],
        "emotion_idx": {"NONE": 0, "HAPPINESS": 1, "FEAR": 2, "SADNESS": 3, "LOVE": 4, "ANGER": 5},
        "weight_decay": 0,
        "learning_rate": 0.0001,
        "seed": 42,
        "save_model": True,
        "load_model": False,
        "save_model_path": "output/models",
        "load_model_path": f"{PROJECT_ROOT}/src/output/models/models/sub_20.pth",
        "output_csv_path": f"{PROJECT_ROOT}/src/output/inference/sub_20.csv"
    }
})

def evaluate_model(model, dataloader, device):
    model.eval()
    predictions_list = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Running Inference on Validation Set"):
            data, labels = batch["data_tensor"], batch["label_tensor"]
            data = data.float().to(device)
            labels = labels.long().to(device)
            if data.dim() == 4:
                data = data.unsqueeze(1)  # Ensure correct input shape
            output = model(data)
            _, predictions = torch.max(output, dim=1)

            # Collect results in list
            for true_label, pred in zip(labels.cpu().numpy(), predictions.cpu().numpy()):
                predictions_list.append([true_label, pred])
    
    return predictions_list

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    train_dataloader, val_dataloader = get_data_loaders(cfg)
    print(f"Loaded Validation Observations: {len(val_dataloader.dataset)}")
    
    output_dim = len(cfg.data.emotion_idx)
    model_path_torch = cfg.data.load_model_path
    
    if cfg.model == "CNN":
        model = CNN(cfg=cfg, output_dim=output_dim)
    elif cfg.model == "ResNet":
        model = ResNet(BasicBlock, [1, 1, 1, 1], in_channels=1, num_classes=output_dim)
    else:
        raise ValueError("Invalid model specified")
    
    if model_path_torch:
        model.load_state_dict(torch.load(model_path_torch, map_location=device))
        print(f"Loaded model from {model_path_torch}")
    
    model.to(device)

    # Run inference
    val_predictions = evaluate_model(model, val_dataloader, device)
    
    # Convert label indices to emotion names
    emotion_idx = cfg.data.emotion_idx
    inverse_emotion_idx = {v: k for k, v in emotion_idx.items()}  # Reverse mapping

    val_predictions_named = [
        [inverse_emotion_idx[true_label], inverse_emotion_idx[predicted]]
        for true_label, predicted in val_predictions
    ]

    # Save to CSV
    output_csv_path = cfg.data.output_csv_path
    df = pd.DataFrame(val_predictions_named, columns=["true_label", "predicted"])
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    df.to_csv(output_csv_path, index=False)
    
    print(f"Validation predictions saved to: {output_csv_path}")

    # Compute per-class accuracy
    total_counts = {k: 0 for k in emotion_idx.keys()}  # Total per-class samples
    correct_counts = {k: 0 for k in emotion_idx.keys()}  # Correct per-class predictions

    for true_label, predicted in val_predictions_named:
        total_counts[true_label] += 1
        if true_label == predicted:
            correct_counts[true_label] += 1

    # Print per-class accuracy
    print("\nValidation Accuracy per Emotion:")
    for emotion, total in total_counts.items():
        if total > 0:
            accuracy = (correct_counts[emotion] / total) * 100
            print(f"{emotion}: {accuracy:.2f}% ({correct_counts[emotion]}/{total})")
        else:
            print(f"{emotion}: No samples in validation set")

    # Print overall validation accuracy
    total_correct = sum(correct_counts.values())
    total_samples = sum(total_counts.values())
    overall_accuracy = (total_correct / total_samples) * 100 if total_samples > 0 else 0
    print(f"\nOverall Validation Accuracy: {overall_accuracy:.2f}% ({total_correct}/{total_samples})")


if __name__ == "__main__":
    main()



Using device: cuda
Dataset contains 8 files.
Spatial dimensions: (132, 175, 48)
Maximum timepoints per file: 542
Subjects: ['sub-20']
Sessions: ['01' '02' '03' '04' '05' '06' '07' '08']
Emotion categories: ['NONE', 'HAPPINESS', 'FEAR', 'SADNESS', 'LOVE', 'ANGER']
Total valid labeled timepoints: 799
Loaded Validation Observations: 160


/tmp/ipykernel_79149/3863842772.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path_torch, map_location=device))


Loaded model from /home/paperspace/DeepEmotion/src/output/models/models/sub_20.pth


Running Inference on Validation Set: 100%|██████████| 8/8 [00:04<00:00,  1.75it/s]

Validation predictions saved to: /home/paperspace/DeepEmotion/src/output/inference/sub_20.csv

Validation Accuracy per Emotion:
NONE: No samples in validation set
HAPPINESS: 67.35% (33/49)
FEAR: 93.33% (28/30)
SADNESS: 77.27% (34/44)
LOVE: 68.42% (13/19)
ANGER: 61.11% (11/18)

Overall Validation Accuracy: 74.38% (119/160)


In [ ]:
import os
import torch
import pandas as pd
from collections import Counter
from tqdm import tqdm
from omegaconf import OmegaConf

# Config variables
PROJECT_ROOT = os.path.abspath("../")

cfg = OmegaConf.create({
    "project_root": PROJECT_ROOT,
    "verbose": True,
    "wandb": True,
    "sys_log": True,
    "model": "CNN",
    "CNN": {
        "c1": 16, "c2": 32, "c3": 64, "k1": 3, "k2": 3, "k3": 3,
        "pk": 2, "ps": 2, "kernel_size": 3, "stride": 1, "padding": 1
    },
    "train": {
        "epochs": 50, "batch_size": 20, "shuffle": True, "train_ratio": 0.8,
        "print_label_frequencies": True
    },
    "data": {
        "data_path": f"{PROJECT_ROOT}/data/raw/derivatives/non-linear_anatomical_alignment",
        "zarr_dir_path": f"{PROJECT_ROOT}/zarr_datasets",
        "zarr_path": f"{PROJECT_ROOT}/zarr_datasets/pool_emotions",
        "label_path": f"{PROJECT_ROOT}/data/updated_annotations/pooled_annotations_structured.tsv",
        "sessions": ["01", "02", "03", "04", "05", "06", "07", "08"],
        "file_pattern_template": "*_ses-forrestgump_task-forrestgump_rec-dico7Tad2grpbold7TadNL_run-{}_bold.nii.gz",
        "subjects": ["sub-20"],
        "session_offsets": [0, 902, 1784, 2660, 3636, 4560, 5438, 6522],
        "emotion_idx": {"NONE": 0, "HAPPINESS": 1, "FEAR": 2, "SADNESS": 3, "LOVE": 4, "ANGER": 5},
        "normalization": False,
        "weight_decay": 0,
        "learning_rate": 0.0001,
        "seed": 42,
        "save_model": True,
        "load_model": False,
        "save_model_path": "output/models",
        "load_model_path": f"{PROJECT_ROOT}/src/output/models/models/sub_20.pth",
        "output_csv_path": f"{PROJECT_ROOT}/src/output/inference/sub_20.csv"
    }
})

def evaluate_model(model, dataloader, device):
    model.eval()
    predictions_list = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Running Inference on Validation Set"):
            data, labels = batch["data_tensor"], batch["label_tensor"]
            data = data.float().to(device)
            labels = labels.long().to(device)
            if data.dim() == 4:
                data = data.unsqueeze(1)  # Ensure correct input shape
            output = model(data)
            _, predictions = torch.max(output, dim=1)

            # Collect results in list
            for true_label, pred in zip(labels.cpu().numpy(), predictions.cpu().numpy()):
                predictions_list.append([true_label, pred])
    
    return predictions_list

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    train_dataloader, val_dataloader = get_data_loaders(cfg)
    print(f"Loaded Validation Observations: {len(val_dataloader.dataset)}")
    
    output_dim = len(cfg.data.emotion_idx)
    model_path_torch = cfg.data.load_model_path
    
    if cfg.model == "CNN":
        model = CNN(cfg=cfg, output_dim=output_dim)
    elif cfg.model == "ResNet":
        model = ResNet(BasicBlock, [1, 1, 1, 1], in_channels=1, num_classes=output_dim)
    else:
        raise ValueError("Invalid model specified")
    
    if model_path_torch:
        model.load_state_dict(torch.load(model_path_torch, map_location=device))
        print(f"Loaded model from {model_path_torch}")
    
    model.to(device)

    # Run inference
    val_predictions = evaluate_model(model, val_dataloader, device)
    
    # Convert label indices to emotion names
    emotion_idx = cfg.data.emotion_idx
    inverse_emotion_idx = {v: k for k, v in emotion_idx.items()}  # Reverse mapping

    val_predictions_named = [
        [inverse_emotion_idx[true_label], inverse_emotion_idx[predicted]]
        for true_label, predicted in val_predictions
    ]

    # Save to CSV
    output_csv_path = cfg.data.output_csv_path
    df = pd.DataFrame(val_predictions_named, columns=["true_label", "predicted"])
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    df.to_csv(output_csv_path, index=False)
    
    print(f"Validation predictions saved to: {output_csv_path}")

    # Compute per-class accuracy
    total_counts = {k: 0 for k in emotion_idx.keys()}  # Total per-class samples
    correct_counts = {k: 0 for k in emotion_idx.keys()}  # Correct per-class predictions

    for true_label, predicted in val_predictions_named:
        total_counts[true_label] += 1
        if true_label == predicted:
            correct_counts[true_label] += 1

    # Print per-class accuracy
    print("\nValidation Accuracy per Emotion:")
    for emotion, total in total_counts.items():
        if total > 0:
            accuracy = (correct_counts[emotion] / total) * 100
            print(f"{emotion}: {accuracy:.2f}% ({correct_counts[emotion]}/{total})")
        else:
            print(f"{emotion}: No samples in validation set")

    # Print overall validation accuracy
    total_correct = sum(correct_counts.values())
    total_samples = sum(total_counts.values())
    overall_accuracy = (total_correct / total_samples) * 100 if total_samples > 0 else 0
    print(f"\nOverall Validation Accuracy: {overall_accuracy:.2f}% ({total_correct}/{total_samples})")


if __name__ == "__main__":
    main()

